In [1]:
from netCDF4 import Dataset
import numpy as np
from pathlib import Path
import sys, getopt
import os
import argparse
from datetime import datetime
from datetime import timedelta
from datetime import date

In [2]:
#Beit Dagan
lat = 34.81
lon = 32.00


In [3]:
#find start lat/lon
def find_start_lat_lon_and_interval(filename):
    
    nc_file = Dataset(filename)
    
    lat_var = nc_file.variables['lat']
    lats_size = len(lat_var)
    lat_start = lat_var[0] if (lat_var[0] < lat_var[lats_size-1]) else lat_var[lats_size-1]
    
    lon_var = nc_file.variables['lon']
    lons_size = len(lon_var)
    lon_start = lon_var[0] if (lon_var[0] < lon_var[lons_size-1]) else lon_var[lons_size-1]
    
    # we assume lats % lons have the same interval 
    interval = (abs(lat_var[0]) + abs(lat_var[lats_size-1]))/lats_size
                
    return(lon_start, lat_start , interval)

In [4]:
def calc_index_by_lon_lat(fileName, lon, lat):
    
    #find start lats & lons
    lon_start, lat_start , interval = find_start_lat_lon_and_interval(filename)
    
    latOffset =  (lat+abs(lat_start))/interval
    lonOffset =  (lon+abs(lon_start))/interval
    
    #get the coordinates values from the file
    nc_file = Dataset(filename)  
    lat_var = nc_file.variables['lat']
    lon_var = nc_file.variables['lon']
    
    #find the cell which is closest to the coordinates we are interested in
    latOffset = latOffset if (((lat - lat_var[latOffset])) < (lat_var[latOffset+1] - lat)) else latOffset
    lonOffset = lonOffset if (((lon - lon_var[lonOffset])) < (lon_var[lonOffset+1] - lon)) else lonOffset
                      
    return(round(lonOffset), round(latOffset))

In [5]:
def get_file_date(fileName):
    #extract last part in the file name which is the date
    filename = os.path.basename(fileName)
    tempTuple = os.path.splitext(filename)
    filename = tempTuple[0]
    fileNameArr = np.array(filename.split("-"))
    filedate = fileNameArr[len(fileNameArr)-1]
    
    #the format is 222001010900Z YYYYMMDDHHMM
    year = int(filedate[:4])
    month = int(filedate[4:6])
    day = int(filedate[6:8])
    hour = int(filedate[8:10])
    min = int(filedate[10:12])
    
    return datetime(year, month, day, hour, min)

In [6]:
def get_file_day_and_hour(fileName):
    #extract last part in the file name which is the date
    filename = os.path.basename(fileName)
    tempTuple = os.path.splitext(filename)
    filename = tempTuple[0]
    fileNameArr = np.array(filename.split("-"))
    filedate = fileNameArr[len(fileNameArr)-1]
    
    #the format is 222001010900Z YYYYMMDDHHMM
    year = int(filedate[:4])
    month = int(filedate[4:6])
    day = int(filedate[6:8])
    hour = int(filedate[8:10])
    minute = int(filedate[10:12])
    
    return (day, hour)

In [7]:
import glob

inputDir = '/Users/alonhrl/Documents/Alon/MA-MSC/development/Eumetsat data/01-22'
#inputDir = '/Users/alonhrl/Documents/Alon/MA-MSC/development/eumetsat'
days_in_month = 31
num_of_radiation_hours = 24
a_shape = (days_in_month+1, num_of_radiation_hours+1)  # we add one so the cell index will match day and hour
monthly_rad = np.zeros(a_shape, dtype=np.int16)
count = 0

#iterate recursively over all netCDF files in the directory
for filename in glob.iglob(inputDir + '**/**', recursive=True):
    file_name, file_extension = os.path.splitext(filename)
    if file_extension == '.nc':
        count +=1
        # Push radiation value into the month array
        fh = Dataset(filename, mode='r')
        ssi = fh.variables['ssi']
        #calculate indices for required lattitude & longitude
        cLon, cLat = calc_index_by_lon_lat(file_name, lon, lat)
        
        
        print("time: %s, lonIndx: %d, latIndx: %d,  ssi: %.2f" %(get_file_date(filename), cLon, cLat, ssi[cLon,cLat]))
        day, hour = get_file_day_and_hour(filename)
                
        #if value is 'nan' set to 0
        if type(ssi[cLon, cLat]) is not np.float64:
            monthly_rad[day][hour] = 0
        else:
            monthly_rad[day][hour] = ssi[cLon, cLat]

#remove first row (no day '0')
monthly_rad = monthly_rad[1:]

print (monthly_rad)        
print ("Processed %d files!" % (count))

time: 2022-01-26 12:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 93.20
time: 2022-01-25 13:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 362.10
time: 2022-01-18 16:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-10 10:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 621.50
time: 2022-01-08 16:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-13 11:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 577.90
time: 2022-01-03 11:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 550.90


/Users/alonhrl/anaconda3/envs/mentor-ai/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Warning: converting a masked element to nan.


time: 2022-01-10 09:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 602.40
time: 2022-01-27 16:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-13 08:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 398.50
time: 2022-01-03 08:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 44.30
time: 2022-01-02 15:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-12 15:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-01 14:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-09 12:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 348.10
time: 2022-01-11 14:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-19 12:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 128.30
time: 2022-01-27 14:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 45.00
time: 2022-01-24 15:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-01 16:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-09 10:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 617.40
time: 2022-01-11 16:00:00, lonIndx: 1840, l

time: 2022-01-22 11:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 621.30
time: 2022-01-31 10:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 698.50
time: 2022-01-29 16:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-21 10:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 664.20
time: 2022-01-14 13:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 86.20
time: 2022-01-04 13:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 96.90
time: 2022-01-07 12:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 455.90
time: 2022-01-20 14:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 40.00
time: 2022-01-28 12:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 138.40
time: 2022-01-30 14:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 176.00
time: 2022-01-23 15:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-06 16:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-16 16:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-31 09:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 351.40
time: 2022-01-21 09:00:00, lonIndx:

time: 2022-01-30 13:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 379.90
time: 2022-01-23 12:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 53.60
time: 2022-01-06 11:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 566.00
time: 2022-01-16 11:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 101.50
time: 2022-01-05 10:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 155.00
time: 2022-01-15 10:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 329.30
time: 2022-01-24 02:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-15 21:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-05 21:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-16 20:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-06 20:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-28 19:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-04 01:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-14 01:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-07 00:00:00, lonIndx: 1840, lat

time: 2022-01-23 20:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-16 23:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-06 23:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-19 08:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 404.80
time: 2022-01-15 22:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-05 22:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-29 07:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 107.60
time: 2022-01-07 03:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-17 03:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-04 02:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-14 02:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-21 03:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-29 05:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-31 03:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-28 18:00:00, lonIndx: 1840, latIndx: 1896,

time: 2022-01-13 18:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-24 07:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 70.30
time: 2022-01-10 19:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-23 17:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-08 22:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-18 22:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-14 08:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 90.30
time: 2022-01-03 01:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-13 01:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-08 06:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 181.00
time: 2022-01-18 06:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 194.00
time: 2022-01-10 00:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-25 03:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-12 21:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-02 21:00:00, lonIndx: 1840, latIndx: 1

time: 2022-01-21 08:00:00, lonIndx: 1840, latIndx: 1896,  ssi: 544.40
time: 2022-01-08 21:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-18 21:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-19 05:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-11 03:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-09 05:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-01 03:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-12 02:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-02 02:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-08 18:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-18 18:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-10 23:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-28 17:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-03 22:00:00, lonIndx: 1840, latIndx: 1896,  ssi: nan
time: 2022-01-13 22:00:00, lonIndx: 1840, latIndx: 1896,  s

In [8]:
np.savetxt("mon_eumetsat_rad-savetxt.csv", monthly_rad, delimiter=",", fmt='%d')
